In [ ]:
working_on_colab = True

if working_on_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  path = "/content/drive/MyDrive/Social Project/"
else:
  path = ""

Mounted at /content/drive


In [ ]:
# Install following packages if you haven't already

!pip install pandas tqdm nltk langdetect scikit-learn top2vec top2vec[sentence_encoders]
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=ccd5e99f5d1fdc7bfbfb742f26f809f0bfd586a50f118ac8e17eb362e947e4f6
  Stored in directory: /root

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from langdetect import detect, DetectorFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from top2vec import Top2Vec
import tensorflow as tf
import torch
import matplotlib.pyplot as plt

## Music Dataset cleaning

In [ ]:
dis_ms_full = pd.read_csv(path+"anon_disorder_musics.csv", index_col=0)

In [ ]:
dis_ms_full

,artist,lyric,created_at,type,title,user_id,src,disorder
0,Echo & the Bunnymen,[Verse 1]\nUnder blue moon I saw you\nSo soon ...,Wed Jan 13 01:02:45 +0000 2021,P,The Killing Moon,4353e884c1,SPOTIFY,anxiety
1,Paul McCartney,NaN,Wed Jan 13 01:02:45 +0000 2021,P,Monkberry Moon Delight - Remastered 2012,4353e884c1,SPOTIFY,anxiety
2,Turin Brakes,"Oooh, oooh, oooh (x2)\nSometimes just letting ...",Wed Jan 13 01:02:45 +0000 2021,P,Red Moon,4353e884c1,SPOTIFY,anxiety
3,Damon Albarn,NaN,Wed Jan 13 01:02:45 +0000 2021,P,"Damon Albarn: Dr Dee, An English Opera: No. 4,...",4353e884c1,SPOTIFY,anxiety
4,Radiohead,[Verse 1]\nI sucked the moon\nI spoke too soon...,Wed Jan 13 01:02:45 +0000 2021,P,Sail To The Moon,4353e884c1,SPOTIFY,anxiety
...,...,...,...,...,...,...,...,...
396837,slenderbodies,Cherry blossom\nSeen from outer space\nRainbow...,Mon Sep 02 01:46:26 +0000 2019,P,cherry blossom,99398d2bec,SPOTIFY,bipolar
396838,Madlib,[Sample]\nNow is the time\nNow is the time\nRi...,Mon Sep 02 01:46:26 +0000 2019,P,Road Of The Lonely Ones,99398d2bec,SPOTIFY,bipolar
396839,slowthai,[Verse]\nFine line between love and hate\nDon'...,Mon Sep 02 01:46:26 +0000 2019,P,Thoughts,99398d2bec,SPOTIFY,bipolar
396840,Tyler Glenn,"[Verse 1]\nYeah, your talk is cheap, you like ...",Fri Jun 07 19:44:29 +0000 2019,P,Shameless,99398d2bec,SPOTIFY,bipolar


Dropping rows that has duplicates in specific rows from the big dataset

In [ ]:
dis_ms_full = dis_ms_full.drop_duplicates(subset=["user_id", "disorder", "artist", "title", "lyric"])

Dropping raws that don't have the lyrics

In [ ]:
dis_ms = dis_ms_full.dropna(subset="lyric")
dis_ms = dis_ms[["user_id", "disorder", "artist", "title", "lyric", "src"]]
dis_ms = dis_ms.dropna()
dis_ms.reset_index(drop=True, inplace=True)
dis_ms

,user_id,disorder,artist,title,lyric,src
0,4353e884c1,anxiety,Echo & the Bunnymen,The Killing Moon,[Verse 1]\nUnder blue moon I saw you\nSo soon ...,SPOTIFY
1,4353e884c1,anxiety,Turin Brakes,Red Moon,"Oooh, oooh, oooh (x2)\nSometimes just letting ...",SPOTIFY
2,4353e884c1,anxiety,Radiohead,Sail To The Moon,[Verse 1]\nI sucked the moon\nI spoke too soon...,SPOTIFY
3,4353e884c1,anxiety,Peace,Under the Moon,[Verse 1]\nIf you call it lonely to be so alon...,SPOTIFY
4,4353e884c1,anxiety,Karen O,The Moon Song - Studio Version Duet,"[Verse 1: Karen O, \nEzra Koenig\n & \nBoth\n]...",SPOTIFY
...,...,...,...,...,...,...
243110,99398d2bec,bipolar,slenderbodies,cherry blossom,Cherry blossom\nSeen from outer space\nRainbow...,SPOTIFY
243111,99398d2bec,bipolar,Madlib,Road Of The Lonely Ones,[Sample]\nNow is the time\nNow is the time\nRi...,SPOTIFY
243112,99398d2bec,bipolar,slowthai,Thoughts,[Verse]\nFine line between love and hate\nDon'...,SPOTIFY
243113,99398d2bec,bipolar,Tyler Glenn,Shameless,"[Verse 1]\nYeah, your talk is cheap, you like ...",SPOTIFY


### Cleaning lyrycs for sentiment and topic analysis

For later analysis i need to apply some transformations to the text of the lyrics.
In particular I'm:
- Detecting if the text is written in english.
- Removing special characters, punctuations or symbols that don't contribute to the analysis.
- Lowercasing (case differences can affect tokenization and analysis).
- Removing stopwords (e.g., "the", "is", "and") that may dominate analysis without adding value.
- Removing more common singer/artist names, colloquialisms and slang, offensive or sensitive terms (for paper purposes)
- Lemmatizing to ensure that variants of the same word (e.g., "running" vs. "run") don't appear as different tokens.

In [ ]:
# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

new_stop_words = ['ooh','yeah','hey','whoa','woah', 'ohh', 'was', 'mmm', 'oooh','yah','yeh','mmm', 'hmm','deh','doh','jah','wa', 'la', "im", "dont", "oh", "ah",
                  "la", "eh", "lalala", "lala", "lalalala", "eh", "nah", "oi", "jingle", "oohoohoohooh", "nana", "ba", "nanananana", "nananana", "nanana",
                  "nana", "na", "hoo", "lyric", "harry", "zayn", "liam", "chester", "shinoda", "bennington", "mike", "oohah", "bom", "bum", "da", "doo", "mi",
                  "ba", "wa", "ni", "ga", "lo", "hmmhmm", "bam", "dadada", "ahhhh", "ohohohoh", "iii", "ii", "wah", "annie", "gabriel", "ooooooh", "ooh", "nae",
                  "nan", "ni", "ga", "ey", "na", 'blah', 'pum', 'ye', 'jo', 'jimin', 'nuh', 'ahha', 'jungkook', 'ahahah', 'oohoh', 'mo', 'nai', 'te', 'wo', 'uhh',
                  'lalalalala', 'lalalalalala', 'sha', 'bop', 'yeahyeah', 'ahh', 'oo', 'pum', 'ohooh', 'mmmmmm', 'oohwhoa', 'youre', 'oooooh', 'ahh', 'ooo', 'ohhhh',
                  'released', 'song', 'lyric', 'que', 'el', 'en', 'mhm', 'em', 'dem', 'di','yuh', 'nuh', 'fi', 'pon', 'inna', 'ohohohohoh', 'ohohohoh', 'ohohoh',
                  'ohoh', 'ohoh', 'bo', 'cha', 'ahhh', 'ahahah', 'ahahahah', 'ahah', 'ohoohoh', 'baba', 'dada', 'mmh', 'let', 'oohoohoohoohooh', 'doodoo', 'dodododo',
                  'oohoohooh', 'wee', 'oohoohoohoohoohooh', 'ohhh', 'oooooooooo', 'heyhey', 'dododo', 'doodoodoo', 'heyhey', 'dodo', 'woo', 'whoo', 'lalale', 'oohooh',
                  'oohoohooh', 'oohoohoohooh', 'oohoohoohoohooh', 'oooo', 'ale', 'aah', 'aha', 'ive', 'aaaah', 'aw', 'vu', 'le', 'woahoh', 'woohoo',
                  "jodeci", "ginuwine", "kehlani", "quavo", "ashanti", "migos", "blackstreet", "carti", "keef", "durk", "mase", "outkast", "thugger", "luda", "swizz", "ugk", "dipset", "jeezy", "ferg",
                  "shawty", "crunk", "finna", "jiggy", "wop", "grindin", "mobb", "fuckboy", "skank", "thugger", "poppin", "ridin", "turnt", "thot", "dougie", "swagga", "ayo", "chea", "jive", "knockin", "dippin", "stuntin",
                  "kkk", "cunt", "nigger", "nigga", "jigga", "ngga", "mothafucka", "whore", "ay", "crackin", "crip", "pappy", "rubberband", "bruise", "바람", "eht", "uoy", "navidad", "dumber", "dadadada",
                  "stuttering", "좋은", "siento", "lag", "cassie", "beez", "peggy", "pokin","hymn", "lullaby", "lyric", "lyrically", "ballad", "jive", "uoy", "crunk", "shawty",
                  "tove", "kurupt", "trina", "ludacris", "tpain", "jeongyeon", "rosanna", "klf", "boosie", "timbaland", "fleek", "hollaback", "dutty", "funkytown", "buzzin", "mobbin", "whack", "jive", "shakin", "cruisin",
                  "syne", "surfin", "runnin", "fixin", "dancin", "tippin", "grime", "poppin", "jumpin", "tipsy", "knack", "crank", "adlibs", "gutta", "thuggin", "rhyme", "stanky", "stoop", "spittin", "slang", "wassup",
                  "nig", "crips", "klan", "fck", "btch", "whore", "dolla", "deadbeat", "mothafucka", "tramp", "cah", "stuttering", "fck", "wack", "fk", "soulja", "worst", "loathing", "dumbing",
                  "jaded", "syne", "whataya", "cherish", "refrain", "yearn", "melancholy"]

stop_words.extend(new_stop_words)
stop_words = set(stop_words)
new_stop_words = set(new_stop_words)

def preprocess_text(text):
    try:
        # Remove unwanted patterns like newlines, section titles, and other markers
        text = re.sub(r'\[.*?\]', '', text)  # Removes things like [Verse 1], [Sample], etc.
        text = re.sub(r'\n', ' ', text)  # Replace newline characters with spaces
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        text = re.sub(r'^\s+|\s+?$', '', text)  # Remove leading and trailing spaces

        # Handle special characters and convert to lowercase
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
        text = text.lower()  # Lowercase the text

        # Tokenization and stopword removal
        tokens = [word for word in text.split() if word not in stop_words and word not in new_stop_words]

        # Lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Join tokens back into a single string
        return ' '.join(tokens)
    except Exception as e:
        return None

def is_english(text):
    try:
        return detect(text) == 'en'
    except Exception as e:
        return False

In [ ]:
chunk_size = 10000
cleaned_data = []

DetectorFactory.seed = 1 # for reproducibility in language detection

for chunk_start in tqdm(range(0, len(dis_ms), chunk_size), desc="Processing chunks"):
    chunk_end = min(chunk_start + chunk_size, len(dis_ms))
    chunk = dis_ms.iloc[chunk_start:chunk_end].copy()

    # Preprocess the lyrics
    chunk['cleaned_lyric'] = chunk['lyric'].apply(preprocess_text)

    # Filter out non-English lyrics
    chunk = chunk[chunk['cleaned_lyric'].apply(is_english)]

    # Drop rows where cleaning failed (e.g., empty or invalid text)
    chunk = chunk.dropna(subset=['cleaned_lyric'])

    # Append the cleaned chunk to the results
    cleaned_data.append(chunk[['user_id', 'disorder', 'artist', 'title', 'cleaned_lyric']])

# Concatenate all cleaned chunks into a single DataFrame
cleaned_data = pd.concat(cleaned_data, ignore_index=True)

# Just to be shure 100%
cleaned_data = cleaned_data.dropna(subset=['cleaned_lyric'])

# Save the cleaned dataset to a new CSV file
cleaned_data.to_csv(path+'cleaned_lyrics_dataset.csv.csv', index=False)

print("Processing complete! Cleaned data saved to 'cleaned_lyrics_dataset.csv'.")

Processing chunks: 100%|██████████| 25/25 [33:15<00:00, 79.80s/it]


Processing complete! Cleaned data saved to 'cleaned_lyrics_dataset.csv'.


Since some words are still in the text I decided to pass the dataset to the clining process a second time

In [ ]:
chunk_size = 10000
final_cleaned_data = []

DetectorFactory.seed = 1 # for reproducibility in language detection

for chunk_start in tqdm(range(0, len(cleaned_data), chunk_size), desc="Processing chunks"):
    chunk_end = min(chunk_start + chunk_size, len(cleaned_data))
    chunk = cleaned_data.iloc[chunk_start:chunk_end].copy()

    # Preprocess the lyrics
    chunk['final_cleaned_lyric'] = chunk['cleaned_lyric'].apply(preprocess_text)

    # Filter out non-English lyrics
    chunk = chunk[chunk['final_cleaned_lyric'].apply(is_english)]

    # Drop rows where cleaning failed (e.g., empty or invalid text)
    chunk = chunk.dropna(subset=['final_cleaned_lyric'])

    # Append the cleaned chunk to the results
    final_cleaned_data.append(chunk[['user_id', 'disorder', 'artist', 'title', 'final_cleaned_lyric']])

# Concatenate all cleaned chunks into a single DataFrame
final_cleaned_data = pd.concat(final_cleaned_data, ignore_index=True)

# Just to be shure 100%
final_cleaned_data = final_cleaned_data.dropna(subset=['final_cleaned_lyric'])

# Save the cleaned dataset to a new CSV file
final_cleaned_data = final_cleaned_data.rename(columns = {'final_cleaned_lyric' : "cleaned_lyric"})
final_cleaned_data.to_csv(path+'final_cleaned_lyrics_dataset.csv', index=False)

print("Processing complete! Cleaned data saved to 'final_cleaned_lyrics_dataset.csv'.")

Processing chunks: 100%|██████████| 23/23 [28:47<00:00, 75.11s/it]


Processing complete! Cleaned data saved to 'final_cleaned_lyrics_dataset.csv'.


Now everything is properly cleaned

In [ ]:
cleaned_data = pd.read_csv(path+"final_cleaned_lyrics_dataset.csv")

In [ ]:
cleaned_data

,user_id,disorder,artist,title,cleaned_lyric
0,4353e884c1,anxiety,Echo & the Bunnymen,The Killing Moon,blue moon saw soon youll take arm late beg can...
1,4353e884c1,anxiety,Turin Brakes,Red Moon,x2 sometimes letting go easier dead friend can...
2,4353e884c1,anxiety,Radiohead,Sail To The Moon,sucked moon spoke soon much cost dropped moonb...
3,4353e884c1,anxiety,Peace,Under the Moon,call lonely alone nobody calling nobody home l...
4,4353e884c1,anxiety,Karen O,The Moon Song - Studio Version Duet,verse 1 karen ezra koenig lying moon dear ill ...
...,...,...,...,...,...
221155,99398d2bec,bipolar,slenderbodies,cherry blossom,cherry blossom seen outer space rainbow patter...
221156,99398d2bec,bipolar,Madlib,Road Of The Lonely Ones,time time right go wrong tell ever treat bad e...
221157,99398d2bec,bipolar,slowthai,Thoughts,fine line love hate know divide truthfully the...
221158,99398d2bec,bipolar,Tyler Glenn,Shameless,talk cheap like mystery awake since 2am barely...


Creating a list of users present in the music dataset

In [ ]:
ms_users = cleaned_data["user_id"].unique()
len(ms_users)

4295

In [ ]:
len(cleaned_data["artist"].unique())

18990

#### Adjacency matrix

In [ ]:
Adj_data = cleaned_data[cleaned_data["user_id"].isin(ms_users)]
Adj_data = Adj_data.drop_duplicates(subset=["user_id", "artist"])
Adj_mat = pd.crosstab(Adj_data["user_id"], Adj_data["artist"])
Adj_mat.to_csv(path+"final_adjacency matrix.csv")
Adj_mat

artist,!!!,"""Weird Al"" Yankovic",$NOT,$WAGGOT,$ean Wire,$tupid Young,$ugar Babies,$uicideBoy$,'68,'Til Tuesday,...,✦ pink cig ✦,アンジェラ (Vo. Alisa),キャロル&チューズデイ(Vo.Nai Br.XX&Celeina Ann),デズモンド(Vo.Marker Starling),マーメイド・シスターズ(Vo.クリヤマユウリ),ヨルシカ,生 Conform 死,秋元薫,박혜진 Park Hye Jin,방탄소년단
user_id,,,,,,,,,,,,,,,,,,,,,
0001e573dc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00021bdb1d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0003c5dea3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00180e338b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001fec300a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffd4a16119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ffddc93808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ffdee350a0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Topic modeling on songs

Since the songs are reoeated for each diffrent user that has a connection with it, before starting the topic modeling I'm creating a dataset with only, one row for each diffrent song

In [ ]:
cleaned_data = pd.read_csv(path+"final_cleaned_lyrics_dataset.csv")

In [ ]:
song_id = cleaned_data[["artist", "title"]].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {"index" : "song_id"})
song_id

,song_id,artist,title
0,0,Echo & the Bunnymen,The Killing Moon
1,1,Turin Brakes,Red Moon
2,2,Radiohead,Sail To The Moon
3,3,Peace,Under the Moon
4,4,Karen O,The Moon Song - Studio Version Duet
...,...,...,...
98541,98541,Jacob Collier,Count The People (feat. Jessie Reyez & T-Pain)
98542,98542,Headie One,Judge Me (Interlude) (feat. FKA twigs)
98543,98543,Jadakiss,ME
98544,98544,Clean Bandit,"24 Hours x Gypsy Woman (feat. Yasmin Green, Cr..."


In [ ]:
cleaned_data_with_id = pd.merge(
    cleaned_data,
    song_id,
    how = "left",
    on=["artist", "title"]
)

cleaned_data_with_id = cleaned_data_with_id[["user_id", "disorder", "song_id", "artist", "title", "cleaned_lyric"]]
cleaned_data_with_id

,user_id,disorder,song_id,artist,title,cleaned_lyric
0,4353e884c1,anxiety,0,Echo & the Bunnymen,The Killing Moon,blue moon saw soon youll take arm late beg can...
1,4353e884c1,anxiety,1,Turin Brakes,Red Moon,x2 sometimes letting go easier dead friend can...
2,4353e884c1,anxiety,2,Radiohead,Sail To The Moon,sucked moon spoke soon much cost dropped moonb...
3,4353e884c1,anxiety,3,Peace,Under the Moon,call lonely alone nobody calling nobody home l...
4,4353e884c1,anxiety,4,Karen O,The Moon Song - Studio Version Duet,verse 1 karen ezra koenig lying moon dear ill ...
...,...,...,...,...,...,...
221155,99398d2bec,bipolar,51387,slenderbodies,cherry blossom,cherry blossom seen outer space rainbow patter...
221156,99398d2bec,bipolar,70555,Madlib,Road Of The Lonely Ones,time time right go wrong tell ever treat bad e...
221157,99398d2bec,bipolar,88472,slowthai,Thoughts,fine line love hate know divide truthfully the...
221158,99398d2bec,bipolar,98545,Tyler Glenn,Shameless,talk cheap like mystery awake since 2am barely...


In [ ]:
df_first_lyrics = cleaned_data_with_id.groupby("song_id").first().reset_index()

ms_for_topic = pd.merge(
    song_id,
    df_first_lyrics[["song_id", "cleaned_lyric"]],
    how = "left",
    on="song_id",
)

ms_for_topic

,song_id,artist,title,cleaned_lyric
0,0,Echo & the Bunnymen,The Killing Moon,blue moon saw soon youll take arm late beg can...
1,1,Turin Brakes,Red Moon,x2 sometimes letting go easier dead friend can...
2,2,Radiohead,Sail To The Moon,sucked moon spoke soon much cost dropped moonb...
3,3,Peace,Under the Moon,call lonely alone nobody calling nobody home l...
4,4,Karen O,The Moon Song - Studio Version Duet,verse 1 karen ezra koenig lying moon dear ill ...
...,...,...,...,...
98541,98541,Jacob Collier,Count The People (feat. Jessie Reyez & T-Pain),come club get spirit loud queenie said king de...
98542,98542,Headie One,Judge Me (Interlude) (feat. FKA twigs),judge judge know shit fuckery broke loved nobo...
98543,98543,Jadakiss,ME,jadakiss babe babe babe get even better baby b...
98544,98544,Clean Bandit,"24 Hours x Gypsy Woman (feat. Yasmin Green, Cr...",24 hour ago sitting home phone got text friend...


#### Using LDA

In [ ]:
# Set parameters
chunk_size = 10000
n_topics = 10
n_top_words = 10
output_file = 'cleaned_lyrics_with_lda_topics.csv'

# Vectorizer and LDA model initialization
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)

# Vectorize the entire dataset
X_tfidf = vectorizer.fit_transform(ms_for_topic['cleaned_lyric'])
print("Step 1 done")

# Fit the LDA model
lda_model.fit(X_tfidf)
print("Step 2 done")

# Extract topics
topic_assignments = lda_model.transform(X_tfidf)
topic_labels = topic_assignments.argmax(axis=1)
print("Step 3 done")

# Get the top words for each of the 10 topics
def get_topic_words(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names_out()
    topic_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[:-n_top_words - 1:-1]
        top_words = [words[i] for i in top_words_idx]
        topic_words[topic_idx] = " ".join(top_words)
    return topic_words

# Get topic words for the model
topic_words = get_topic_words(lda_model, vectorizer, n_top_words)
print("Step 4 done")

# Add topic information back to the original dataset
ms_for_topic['topic'] = None
ms_for_topic['topic_description'] = None

# Update the rows with valid lyrics
valid_lyrics_indices = ms_for_topic['cleaned_lyric'].index
ms_for_topic.loc[valid_lyrics_indices, 'topic'] = topic_labels
ms_for_topic.loc[valid_lyrics_indices, 'topic_description'] = ms_for_topic.loc[valid_lyrics_indices, 'topic'].map(topic_words)

# Save the final result
ms_for_topic[['song_id', 'artist', 'title', 'cleaned_lyric', 'topic', 'topic_description']].to_csv(path+output_file, index=False)
print("Topic modeling complete! Data with topics saved to 'cleaned_lyrics_with_topics.csv'.")

Step 1 done
Step 2 done
Step 3 done
Step 4 done
Topic modeling complete! Data with topics saved to 'cleaned_lyrics_with_topics.csv'.


In [ ]:
ms_for_topic

,song_id,artist,title,cleaned_lyric,topic,topic_description
0,0,Echo & the Bunnymen,The Killing Moon,blue moon saw soon youll take arm late beg can...,2,love like light come eye away know ill time night
1,1,Turin Brakes,Red Moon,x2 sometimes letting go easier dead friend can...,2,love like light come eye away know ill time night
2,2,Radiohead,Sail To The Moon,sucked moon spoke soon much cost dropped moonb...,2,love like light come eye away know ill time night
3,3,Peace,Under the Moon,call lonely alone nobody calling nobody home l...,9,love know world like beautiful want life time ...
4,4,Karen O,The Moon Song - Studio Version Duet,verse 1 karen ezra koenig lying moon dear ill ...,2,love like light come eye away know ill time night
...,...,...,...,...,...,...
98541,98541,Jacob Collier,Count The People (feat. Jessie Reyez & T-Pain),come club get spirit loud queenie said king de...,2,love like light come eye away know ill time night
98542,98542,Headie One,Judge Me (Interlude) (feat. FKA twigs),judge judge know shit fuckery broke loved nobo...,8,know love time say like think want thing cause...
98543,98543,Jadakiss,ME,jadakiss babe babe babe get even better baby b...,3,like got aint shit man know fuck ya yo thats
98544,98544,Clean Bandit,"24 Hours x Gypsy Woman (feat. Yasmin Green, Cr...",24 hour ago sitting home phone got text friend...,8,know love time say like think want thing cause...


In [ ]:
ms_for_topic['topic_description'].unique()

array(['love like light come eye away know ill time night',
       'love know world like beautiful want life time way day',
       'girl baby know wanna love like got want gonna right',
       'bitch got aint like shit fuck ayy know money gon',
       'love baby want know come feel wanna dance like make',
       'like got aint shit man know fuck ya yo thats',
       'know love time say like think want thing cause said',
       'know love ill feel time away heart like want wont',
       'lord god jesus holy heaven hallelujah praise glory sing king',
       'check christmas merry santa snow bell year sleigh bye holiday'],
      dtype=object)

Using LDA the process is really slow and the results are not great at all. Topics are overlapping and you can't conclude nothing.

#### Using Top2Vec

In [ ]:
# Parameters
n_topics = 20
output_file_name = 'songs_topics.csv'

# Setting seed
tf.random.set_seed(123)
torch.manual_seed(123)

# Model
cleaned_lyrics = ms_for_topic['cleaned_lyric'].dropna().tolist()

print("Training Top2Vec model...")
top2vec_model = Top2Vec(cleaned_lyrics, embedding_model='universal-sentence-encoder') # I'm starting from a pretrained model
print("Step 1 done")

print(f"Reducing topics to {n_topics}...")
top2vec_model.hierarchical_topic_reduction(num_topics=n_topics)
print("Step 2 done")

print("Extracting topic assignments...")
song_topics = top2vec_model.get_documents_topics(doc_ids = list(ms_for_topic["song_id"]), num_topics = 1, reduced = True)[0]
topics_words = top2vec_model.get_documents_topics(doc_ids = list(ms_for_topic["song_id"]), num_topics = 1, reduced = True)[2]
topics_words = np.array([' '.join(lst) for lst in topics_words])
print("Step 3 done")

print("Adding topics to the dataset...")
final_topics = ms_for_topic.copy()
final_topics['topic'] = song_topics
final_topics['topic_description'] = topics_words

# Save the final dataset
final_topics.to_csv(path+output_file_name, index=False)
print(f"Topic modeling complete! Data with topics saved to '{path+output_file_name}'.")

Training Top2Vec model...


2025-01-21 17:18:08,992 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2025-01-21 17:18:55,906 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2025-01-21 17:19:01,982 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2025-01-21 17:20:56,722 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-01-21 17:23:32,505 - top2vec - INFO - Finding de

Step 1 done
Reducing topics to 20...


Reducing topics: 100%|██████████| 218/218 [02:31<00:00,  1.44it/s]


Step 2 done
Extracting topic assignments...
Step 3 done
Adding topics to the dataset...
Topic modeling complete! Data with topics saved to '/content/drive/MyDrive/Social Project/songs_topics.csv'.


This was much faster

In [ ]:
final_topics

,song_id,artist,title,cleaned_lyric,topic,topic_description
0,0,Echo & the Bunnymen,The Killing Moon,blue moon saw soon youll take arm late beg can...,8,scream commotion morbid slither dyin forsaken ...
1,1,Turin Brakes,Red Moon,x2 sometimes letting go easier dead friend can...,6,fallin raindrop runaway moonlit yearning loves...
2,2,Radiohead,Sail To The Moon,sucked moon spoke soon much cost dropped moonb...,19,rhyming raindrop oar undertow reeling poem qui...
3,3,Peace,Under the Moon,call lonely alone nobody calling nobody home l...,1,commotion reeling lovesick heartbreaker runawa...
4,4,Karen O,The Moon Song - Studio Version Duet,verse 1 karen ezra koenig lying moon dear ill ...,6,fallin raindrop runaway moonlit yearning loves...
...,...,...,...,...,...,...
98541,98541,Jacob Collier,Count The People (feat. Jessie Reyez & T-Pain),come club get spirit loud queenie said king de...,0,rhyming hippopotamus bumpin creole milli kno d...
98542,98542,Headie One,Judge Me (Interlude) (feat. FKA twigs),judge judge know shit fuckery broke loved nobo...,5,lovesick amour amor heartache luv loveless bro...
98543,98543,Jadakiss,ME,jadakiss babe babe babe get even better baby b...,0,rhyming hippopotamus bumpin creole milli kno d...
98544,98544,Clean Bandit,"24 Hours x Gypsy Woman (feat. Yasmin Green, Cr...",24 hour ago sitting home phone got text friend...,9,commotion loveless runaway reckon hoped sleepw...


In [ ]:
ms_data_with_topics = pd.merge(
    cleaned_data_with_id,
    final_topics[["song_id", "topic", "topic_description"]],
    how = "left",
    on="song_id"
)

ms_data_with_topics.to_csv(path+"final_cleaned_lyrics_with_topics.csv", index=False)


In [ ]:
for topic in ms_data_with_topics["topic"].unique():
    top2vec_model.generate_topic_wordcloud(topic, reduced = True)
    plt.savefig(path+"Songs_wordclouds/" + str(topic) +"_wordcloud.png")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
ms_data_with_topics["topic_description"].unique()

array(['scream commotion morbid slither dyin forsaken gruesome loveless sadistic bleed wretched composure nevermore hopelessness motionless gash heartache exile whispered hurt cry disturbed affliction carelessly shall hoped antidote anguish demise rhyming brokenhearted die wail screamed dismay lull pacify cryin suck withered unsaid darkest cynic severed haunt bleeds heartbreaker corpse bury heartless',
       'fallin raindrop runaway moonlit yearning lovesick leavin kaleidoscope loveless gloom wrld hazy twinkle daybreak sunrise melody reeling howling vertigo serenity poem losin interlude daffodil firework sparkle foreigner evermore dyin exile motionless slippin premonition indecision poetic lyrical daze moonlight rainin forecast tune shimmering stillness fleeting landslide cryin moonchild sunshine reckon strobe',
       'rhyming raindrop oar undertow reeling poem quicksand drowns leavin sunk abyss interlude gully plunder blowin sail levee cyclone rainin lyrical landslide ashore drown r

Also the results seems pretty good. Now I'll try to categorize each topic with a title and add it to the dataset before saving it for the last time

In [ ]:
topic_names = {
    8 : "Morbid Heartache",
    6 : "Lovesick Melody",
    19: "Sinking Lullaby",
    1: "Lovesick Chaos",
    3: "Lover's Groove",
    0 : "Playful Beats",
    11 : "Heartache Groove",
    12 : "Dancefloor Vibes",
    10 : "Bumping Rhythms",
    2 : "Illest Flow",
    14 : "Lying Heart",
    4 : "Heartbreaker Vibes",
    7 : "Goodbye Love",
    18 : "Heartache Lament",
    13 : "Desire's Call",
    16 : "Baby's Love",
    5 : "Amorous Yearning",
    15 : "Forsaken Rhymes",
  17 : "Heavenly Devotion",
  9 : "Loveless Escape"
}

In [ ]:
ms_data_with_topics['topic_name'] = ms_data_with_topics['topic'].map(topic_names)
ms_data_with_topics = ms_data_with_topics[["user_id", "disorder", "song_id", "artist", "title", "cleaned_lyric", "topic", "topic_name", "topic_description"]]
ms_data_with_topics

,user_id,disorder,song_id,artist,title,cleaned_lyric,topic,topic_name,topic_description
0,4353e884c1,anxiety,0,Echo & the Bunnymen,The Killing Moon,blue moon saw soon youll take arm late beg can...,8,Morbid Heartache,scream commotion morbid slither dyin forsaken ...
1,4353e884c1,anxiety,1,Turin Brakes,Red Moon,x2 sometimes letting go easier dead friend can...,6,Lovesick Melody,fallin raindrop runaway moonlit yearning loves...
2,4353e884c1,anxiety,2,Radiohead,Sail To The Moon,sucked moon spoke soon much cost dropped moonb...,19,Sinking Lullaby,rhyming raindrop oar undertow reeling poem qui...
3,4353e884c1,anxiety,3,Peace,Under the Moon,call lonely alone nobody calling nobody home l...,1,Lovesick Chaos,commotion reeling lovesick heartbreaker runawa...
4,4353e884c1,anxiety,4,Karen O,The Moon Song - Studio Version Duet,verse 1 karen ezra koenig lying moon dear ill ...,6,Lovesick Melody,fallin raindrop runaway moonlit yearning loves...
...,...,...,...,...,...,...,...,...,...
221155,99398d2bec,bipolar,51387,slenderbodies,cherry blossom,cherry blossom seen outer space rainbow patter...,6,Lovesick Melody,fallin raindrop runaway moonlit yearning loves...
221156,99398d2bec,bipolar,70555,Madlib,Road Of The Lonely Ones,time time right go wrong tell ever treat bad e...,14,Lying Heart,lie deceive unsaid lied dire truthful told lyi...
221157,99398d2bec,bipolar,88472,slowthai,Thoughts,fine line love hate know divide truthfully the...,0,Playful Beats,rhyming hippopotamus bumpin creole milli kno d...
221158,99398d2bec,bipolar,98545,Tyler Glenn,Shameless,talk cheap like mystery awake since 2am barely...,4,Heartbreaker Vibes,rhyming jukebox heartbreaker takin choppa ridd...


In [ ]:
ms_data_with_topics.to_csv(path+"final_cleaned_lyrics_with_topics.csv", index=False)

## Twitter data

Now I'll clean the other dataset in a really similar way I did for the songs.

The diffrences are that I'll rely only on default english stopwords, and I'll remove also things like Urls and @mentions, while keeping the text of the hashtags (without the "#" character) as they can be useful for topic modeling

In [ ]:
dis_tw_full = pd.read_csv("anon_disorder_tweets.csv", index_col=0)

In [ ]:
dis_tw_full

,user_id,text,disorder
0,3f21058fc8,@amirulmokhtar Exactly! We were busy buat untu...,anxiety
1,3f21058fc8,Ternangis baca text &amp; dengar call reply di...,anxiety
2,3f21058fc8,I learn smthg very valuable today.\n\nWhen I w...,anxiety
3,3f21058fc8,"@MintKr Ohhh okay. So, to create more opportun...",anxiety
4,3f21058fc8,@MintKr Perspective expanding experience tu ap...,anxiety
...,...,...,...
32191363,c789b769d1,Peeps Don't forget to catch the braai show @SA...,bipolar
32191364,c789b769d1,@JohnnyGargano #fortheloveofGarganoway🔥💝,bipolar
32191365,c789b769d1,@Official_SABC1 @akaworldwide @malusi_gigaba D...,bipolar
32191366,c789b769d1,#TeamBreezyForever🔥💝💝😇#TeamBreezySA 🇿🇦💝#ChrisB...,bipolar


In [ ]:
dis_tw = dis_tw_full[dis_tw_full["user_id"].isin(ms_users)]
dis_tw.reset_index(drop=True, inplace=True)
dis_tw

,user_id,text,disorder
0,3f21058fc8,@amirulmokhtar Exactly! We were busy buat untu...,anxiety
1,3f21058fc8,Ternangis baca text &amp; dengar call reply di...,anxiety
2,3f21058fc8,I learn smthg very valuable today.\n\nWhen I w...,anxiety
3,3f21058fc8,"@MintKr Ohhh okay. So, to create more opportun...",anxiety
4,3f21058fc8,@MintKr Perspective expanding experience tu ap...,anxiety
...,...,...,...
8757649,d399c5c6ad,They thought I had food https://t.co/4tNQ22gRSX,bipolar
8757650,d399c5c6ad,On my walk today https://t.co/NvEW4tyIxe,bipolar
8757651,d399c5c6ad,My tiny powerful statue that came today https:...,bipolar
8757652,d399c5c6ad,@mindfulheal Its those moments that make me fi...,bipolar


In [ ]:
del dis_tw_full

In [ ]:
# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_tweet(text):
    try:
        # Ensure the input is a valid string
        if not isinstance(text, str):
            return None

        text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
        text = re.sub(r'@\w+', '', text)  # Remove @mentions

        # Handleing hashtags
        hashtags = re.findall(r'#\w+', text)
        text = re.sub(r'#\w+', ' '.join(hashtags), text)

        text = re.sub(r'[^\w\s]', '', text) # Remove punctuation and special characters
        text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
        text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space

        # Lowercasing
        text = text.lower()

        # Tokenization and stopword removal
        tokens = [word for word in text.split() if word not in stop_words]

        # Lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Join tokens back into a single string
        cleaned_text = ' '.join(tokens)

        return cleaned_text
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

def is_english(text):
    try:
        detected_lang = detect(text)
        return detected_lang == 'en'
    except Exception as e:
        return False

In [ ]:
chunk_size = 10000  # Processing the dataset in chunks to save memory
cleaned_tweets_data = []

DetectorFactory.seed = 42

for chunk_start in tqdm(range(0, len(dis_tw), chunk_size), desc="Processing chunks"):
    chunk_end = min(chunk_start + chunk_size, len(dis_tw))
    chunk = dis_tw.iloc[chunk_start:chunk_end].copy()

    # Preprocess the tweets
    chunk['cleaned_text'] = chunk['text'].apply(preprocess_tweet)

    # Filter out non-English tweets
    chunk = chunk[chunk['cleaned_text'].apply(is_english)]

    # Drop rows where cleaning failed (e.g., empty or invalid text)
    chunk = chunk.dropna(subset=['cleaned_text'])

    # Append the cleaned chunk to the results
    cleaned_tweets_data.append(chunk[['user_id', 'disorder', 'cleaned_text']])

# Concatenate all cleaned chunks into a single DataFrame
cleaned_tweets_data = pd.concat(cleaned_tweets_data, ignore_index=True)

# Save the cleaned dataset to a new CSV file
cleaned_tweets_data.to_csv(path+'cleaned_tweets_dataset.csv', index=False)

print("Processing complete! Cleaned tweet data saved to 'cleaned_tweets_dataset.csv'.")

Processing chunks: 100%|██████████| 876/876 [12:16:57<00:00, 50.48s/it]  


Processing complete! Cleaned tweet data saved to 'cleaned_tweets_dataset.csv'.


In [ ]:
cleaned_tweets_data

,user_id,disorder,cleaned_text
0,3f21058fc8,anxiety,ternangis baca text amp dengar call reply dieo...
1,3f21058fc8,anxiety,learn smthg valuable today honest condition cl...
2,3f21058fc8,anxiety,ohhh okay create opportunity thing engaging pe...
3,3f21058fc8,anxiety,pricelesshaha
4,3f21058fc8,anxiety,couldnt sleep figured need stimulus win anxiet...
...,...,...,...
4988652,d399c5c6ad,bipolar,sometimes feel little guilty dumb praying jesu...
4988653,d399c5c6ad,bipolar,find simple word someone say add fuel internal...
4988654,d399c5c6ad,bipolar,thought food
4988655,d399c5c6ad,bipolar,tiny powerful statue came today


### Topic modeling on twitter data

Since the twitter dataset is too big for a topic modeling of the full dataset, I'm focusing on a random sample.
On the selection I'm making shure that each user has a good amount of data included.

The rest of the process is basically the same one it's been done for the songs, except I'll use only Top2Vec library since it was the one working the best on this amount of data.

In [ ]:
cleaned_tweets_data = pd.read_csv(path+"cleaned_tweets_dataset.csv")

In [ ]:
# Calculate the number of lines to sample per user
desired_sample_size = 150000
total_users = cleaned_tweets_data['user_id'].nunique()
lines_per_user = max(1, desired_sample_size // total_users)

# Sample data
sampled_tweets = cleaned_tweets_data.groupby('user_id', group_keys=False).apply(lambda group: group.sample(min(len(group), lines_per_user), random_state=1))

sampled_tweets.reset_index(inplace=True, drop=True)

sampled_tweets

<ipython-input-8-1f2d1662269b>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_tweets = cleaned_tweets_data.groupby('user_id', group_keys=False).apply(lambda group: group.sample(min(len(group), lines_per_user), random_state=1))


,user_id,disorder,cleaned_text
0,0001e573dc,anxiety,love important announcement twitter
1,0001e573dc,anxiety,right mind decides business three year ffs lif...
2,0001e573dc,anxiety,good good luck english lit paper 2 x
3,0001e573dc,anxiety,really light blonde
4,0001e573dc,anxiety,okay least know dont labour government wake mo...
...,...,...,...
145649,fff927bc35,depression,know wrote
145650,fff927bc35,depression,im going cry hell
145651,fff927bc35,depression,isnt tshirt also queer brand
145652,fff927bc35,depression,wait im slow know boot harrys shoe right louis


In [ ]:
n_topics = 20
output_file_name = 'final_twitter_with_topics.csv'

# Setting seed
tf.random.set_seed(123)
torch.manual_seed(123)

# Model
cleaned_tweets = sampled_tweets['cleaned_text'].dropna().tolist()

print("Training Top2Vec model...")
top2vec_model = Top2Vec(cleaned_tweets, embedding_model='universal-sentence-encoder')
print("Step 1 done")

print(f"Reducing topics to {n_topics}...")
top2vec_model.hierarchical_topic_reduction(num_topics=n_topics)
print("Step 2 done")

print("Extracting topic assignments...")
twitts_topics = top2vec_model.get_documents_topics(doc_ids = list(range(len(cleaned_tweets))), num_topics = 1, reduced = True)[0]
topics_words = top2vec_model.get_documents_topics(doc_ids = list(range(len(cleaned_tweets))), num_topics = 1, reduced = True)[2]
topics_words = np.array([' '.join(lst) for lst in topics_words])
print("Step 3 done")

print("Adding topics to the dataset...")
sampled_tweets['topic'] = twitts_topics
sampled_tweets['topic_description'] = topics_words

# Save the final dataset
sampled_tweets.to_csv(path+output_file_name, index=False)
print(f"Topic modeling complete! Data with topics saved to '{path+output_file_name}'.")

2025-01-21 20:09:40,644 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training


Training Top2Vec model...


2025-01-21 20:09:49,272 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2025-01-21 20:09:59,291 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2025-01-21 20:10:48,228 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-01-21 20:15:16,047 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/de

Step 1 done
Reducing topics to 20...


Reducing topics: 100%|██████████| 996/996 [17:57<00:00,  1.08s/it]


Step 2 done
Extracting topic assignments...
Step 3 done
Adding topics to the dataset...
Topic modeling complete! Data with topics saved to '/content/drive/MyDrive/Social Project/final_twitter_with_topics.csv'.


In [ ]:
sampled_tweets

,user_id,disorder,cleaned_text,topic,topic_description
0,0001e573dc,anxiety,love important announcement twitter,19,tweeting twitter tweeted tweet unfollow retwee...
1,0001e573dc,anxiety,right mind decides business three year ffs lif...,13,fuck crap barely shitty boring damn fucked suc...
2,0001e573dc,anxiety,good good luck english lit paper 2 x,11,happy merry happiest congratulation wishing co...
3,0001e573dc,anxiety,really light blonde,17,wig haircut lip makeup sweater hoodie dress ha...
4,0001e573dc,anxiety,okay least know dont labour government wake mo...,7,song rap spotify tune hiphop music soundcloud ...
...,...,...,...,...,...
145649,fff927bc35,depression,know wrote,6,thanks wow ty ahhhh thankyou ahhh great ahh th...
145650,fff927bc35,depression,im going cry hell,15,asleep tired depressed cry fuck sleep sobbing ...
145651,fff927bc35,depression,isnt tshirt also queer brand,17,wig haircut lip makeup sweater hoodie dress ha...
145652,fff927bc35,depression,wait im slow know boot harrys shoe right louis,10,josh drew justin spiderman shawn tyler charlie...


In [ ]:
sampled_tweets["topic_description"].unique()

array(['tweeting twitter tweeted tweet unfollow retweet hashtag unfollowed insta instagram trending emoji shoutout soundcloud follower facebook fam lmaoooo tumblr ig selfie followed celebrity smh snapchat lmfao stfu fanart lowkey hmu stalker niall fandom lmao hahahaha fb adore bot cancelled irl apology motherfucker hilarious fuck lol hahaha asshole petition comeback merch',
       'fuck crap barely shitty boring damn fucked suck fuckin fail smh motherfucker sigh rant nope lazy rip ugh oops hella never reminder repost pathetic fucking lame ew shit ruined quit sad yikes dang dumbass lately idc idiot nah regret omg hahaha lol depressed failed pissed someday awkward awful troll hahahaha',
       'happy merry happiest congratulation wishing congrats happier friday glad yay celebrating celebrate awww birthday thankful goodnight enjoy wonderful excited lucky omg sweetest monday bday celebration cutest wednesday hehe tonight tuesday thursday blessed fantastic badass aww yessss proud weekend en

In [ ]:
for topic in sampled_tweets["topic"].unique():
    top2vec_model.generate_topic_wordcloud(topic, reduced = True)
    plt.savefig(path+"Twitter_wordclouds/" + str(topic) +"_wordcloud.png")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
topic_names = {
    19 : "Social media",
    13 : "Frustration and Disappointment",
    11 : "Happy congratulations",
    17 : "Fashion Choices",
    7 : "Music",
    6 : "Sorprise",
    15 : "Emotional Exhaustion",
    12 : "Charming Emotions",
    0 : "Chaotic Frustration",
    8 : "Entertainment",
    3 : "Emotional Rollercoaster",
    14 : "Hopeful Desires",
    4 : "Comical Annoyances",
    10 : "Celebrities",
    9 : "Eating Food",
    18 : "Mental Struggles",
    2 : "Anger Frustration",
    16 : "Relationships and Labels",
    1 : "General Insults",
    5 : "Kpop Fandom"
}

In [ ]:
sampled_tweets['topic_name'] = sampled_tweets['topic'].map(topic_names)
sampled_tweets = sampled_tweets[["user_id", "disorder", "cleaned_text", "topic", "topic_name", "topic_description"]]
sampled_tweets

,user_id,disorder,cleaned_text,topic,topic_name,topic_description
0,0001e573dc,anxiety,love important announcement twitter,19,Social media,tweeting twitter tweeted tweet unfollow retwee...
1,0001e573dc,anxiety,right mind decides business three year ffs lif...,13,Frustration and Disappointment,fuck crap barely shitty boring damn fucked suc...
2,0001e573dc,anxiety,good good luck english lit paper 2 x,11,Happy congratulations,happy merry happiest congratulation wishing co...
3,0001e573dc,anxiety,really light blonde,17,Fashion Choices,wig haircut lip makeup sweater hoodie dress ha...
4,0001e573dc,anxiety,okay least know dont labour government wake mo...,7,Music,song rap spotify tune hiphop music soundcloud ...
...,...,...,...,...,...,...
145649,fff927bc35,depression,know wrote,6,Sorprise,thanks wow ty ahhhh thankyou ahhh great ahh th...
145650,fff927bc35,depression,im going cry hell,15,Emotional Exhaustion,asleep tired depressed cry fuck sleep sobbing ...
145651,fff927bc35,depression,isnt tshirt also queer brand,17,Fashion Choices,wig haircut lip makeup sweater hoodie dress ha...
145652,fff927bc35,depression,wait im slow know boot harrys shoe right louis,10,Celebrities,josh drew justin spiderman shawn tyler charlie...


In [ ]:
sampled_tweets.to_csv(path+"final_twitter_with_topics.csv", index=False)